# T3Bench for Enhancer Evaluation

Please follow this notebook carefully to evaluate our enhancers for 3D generation.

To run this locally, the following environment has been found to work well:

- Python 3.10
- Torch 2.3.0
- CUDA 12.1
- C++17 support

#### Note: For a given run, save the video file from "gsgen/checkpoints/{prompt}/{date}/{uid}/eval/video" or save the GIF from Wandb and convert it to an MP4. This file will be needed for evaluation

### For T3Bench, you will need a HuggingFace account and an OpenAI API account

# **Utilities [IMPORTANT]**

First we need to change the Colab Python version:

1. Run the following cell
2. Refresh the page
3. Ensure python runtime is py310

In [ ]:
!wget -O py310.sh https://raw.githubusercontent.com/j3soon/colab-python-version/main/scripts/py310.sh
!bash py310.sh

If this has been done successfully, this should read something like:

`User Current Version:- 3.10.16 (main, Dec 11 2024, 16:24:50) [GCC 11.2.0]`


In [ ]:
import sys
print("User Current Version:-", sys.version)

Now run the following once to download the correct CUDA version and add to toolchain

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/12.1.1/local_installers/cuda_12.1.1_530.30.02_linux.run
!sudo sh cuda_12.1.1_530.30.02_linux.run --silent --toolkit --no-opengl-libs
import os

os.environ["PATH"] += ":/usr/local/cuda-12.1/bin"
os.environ["LD_LIBRARY_PATH"] += ":/usr/local/cuda-12.1/lib64"
os.environ["CUDA_HOME"] = "/usr/local/cuda-12.1"

!nvcc --version

# T3Bench Setup

Now you will need to clone the T3Bench fork, or simply add the project to the colab file system in the path `/content/`.

In [ ]:
!rm -rf T3Bench/
!git clone https://github.com/kenL4/T3Bench.git

**[IMPORTANT]**

There is a known [issue](https://github.com/THU-LYJ-Lab/T3Bench/issues/6), where salesforce-lavis and imagereward have directly conflicting package requirements so they are incompatible. The workaround in my fork is to remove salesforce-lavis from the requirements and then install it directly afterwards with `--no-deps` enabled.

Colab will throw a warning that you must restart the runtime but you can safely ignore this warning.

In [ ]:
!pip install -r T3Bench/requirements.txt
!pip install tokenizers --no-cache-dir
!pip install fairscale==0.4.4 timm==0.4.12 salesforce-lavis --no-deps
!pip install "transformers<4.56" sacremoses==0.1.0 "tokenizers<0.22" -U --no-deps --no-cache-dir

# Prepare for evaluation

Only once you have followed these steps should you run the T3Bench scripts:
1. First, you should replace the prompts in T3Bench/data/prompt_single.txt with the prompts you want to evaluate.
2. Move the desired video file(s) to "T3Bench/outputs_mesh_t3/gsgen_single/{prompt}/eval.mp4"

- If you need to convert GIF to MP4, you can use this shell command

In [ ]:
!mkdir -p "T3Bench/outputs_video/gsgen_single/A_cactus_with_pink_flowers/"
!ffmpeg -i "cactus.gif" -movflags faststart -pix_fmt yuv420p "T3Bench/outputs_video/gsgen_single/A_cactus_with_pink_flowers/eval.mp4"

# Evaluate Quality

Run the following cell to compute the quality metric of each prompt in "T3Bench/data/prompt_single.txt"

In [ ]:
# Quality specific versions
!pip install "numpy<2" "huggingface_hub==0.25.2" diffusers transformers opencv-python clip -U

!cd T3Bench/ && \
python run_eval_quality.py --group single --gpu 0 --method gsgen

# Evaluate Alignment

**[IMPORTANT]**

From here on out, you will need an OpenAI API key, so **please replace the API key in run_caption.py and run_eval_alignment.py**

Then, you can run the follwoing commands

In [ ]:
# Alignment specific packages
!pip install decord openai einops --upgrade
!pip install "transformers==4.30.2" "tokenizers<0.14" -U --no-deps --no-cache-dir

# For our limited VRAM, use a smaller model
!sed -i 's/pretrain_flant5xxl/pretrain_flant5xl/g' "T3Bench/run_caption.py"

!rm -rf outputs_caption
!cd T3Bench/ && \
python3.10 run_caption.py --group single --gpu 0 --method gsgen
!cd T3Bench/ && \
python run_eval_alignment.py --group single --gpu 0 --method gsgen

# T3Bench Utils
If you want to clear the model cache, you can use this

In [ ]:
# Set to True or delete if clause to run
if False:
  import shutil
  import os

  cache_dirs = [
      os.path.expanduser("/root/.cache/lavis"),
      os.path.expanduser("/root/.cache/huggingface/hub"),
      os.path.expanduser("/root/.cache/torch/hub/checkpoints"),
  ]

  for d in cache_dirs:
      if os.path.exists(d):
          shutil.rmtree(d)
          print(f"Removed cache: {d}")
      else:
          print(f"Cache not found: {d}")
